In [2]:
import morePVs_output as opm
import morePVs as mpv
import os
import en_utilities as um
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

Plotting Total Building Costs
-----------------------------------


Repeating barchart plot from APSRC 2017 paper
1) for EN
2) for btm
For range of:
* PV sizes
* external tariffs and amortization periods

A) Repeat plot, replot for other parameters

__Now: plot chart for each site with ALL sceanrios__
* Tricky bit is the labelling

* Later - use combined results, and iterate thru sites.
* For now use single site (but also iterate):

   
   
 


In [3]:
#
# Need to set this up using combined_value4 as the study
# value4 is en arrangements only
project='EN1_value_of_pv'
study_name='siteJ_value4'
base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\studies'
op = opm.Output(base_path = base_path,
                          project = project,
                          study_name = study_name)

In [4]:
self = op
df = self.data.copy()
df_in = self.study_parameters.copy()
df = df.merge(df_in,left_index = True, right_index=True)
for c in df.columns:
    if '_x' in c:
        nc = c[:-2]
        df.rename(columns={c:nc}, inplace=True)

In [5]:
# Set up constants
sites=['A','E','D','B','H','I','G','C','J','F']

# get pv sizes for maximum pv systems
maximums={}
pv_ref_file='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\reference\\capex_pv_lookup.csv'
pv_ref=pd.read_csv(pv_ref_file)
pv_ref = pv_ref.set_index('pv_cap_id')
for i in pv_ref.index:
    if 'max' in i and 'site' in i:
        site = um.find_between(i,'_','_')
        maximums[site] = pv_ref.loc[i,'kW']
en_capexes= ['capex_4','capex_5','capex_6']
parents = ['EA310_12c_FIT8', 'EA310_12c_FIT12', 'EA310_12c',
           'EA310_10c_FIT8', 'EA310_10c_FIT12', 'EA310_10c',
         'EA310_8c_FIT8', 'EA310_8c_FIT12','EA310_8c']
a_rates =[5,10,15,20,25]

In [6]:
# get kWp / unit and total $ / unit for each scenario
df['pv_filename'].fillna('zero',inplace=True)
for s in df.index:
    site = df.loc[s,'load_folder'][-1]
    df.loc[s,'site']=site
    if df.loc[s,'pv_filename'] == 'zero':
        df.loc[s,'kwp_unit'] =0
    elif 'max' in df.loc[s,'pv_filename']:
        df.loc[s,'kwp_unit']= maximums[site]/df.loc[s,'number_of_households']
    else:
        df.loc[s,'kwp_unit'] = float(df.loc[s,'pv_filename'][-9]) + float(df.loc[s,'pv_filename'][-7])/10
    df.loc[s,'$_unit'] = df.loc[s,'total$_building_costs_mean']/df.loc[s,'number_of_households']

    
        

In [7]:
# Organise labesl 
df.loc[:,'site'] = df.loc[:,'load_folder'].apply(lambda x : x[-1])
df.loc[:, 'labels'] = df.loc[:,'site']
floors = {'A':12,'E':7, 'D':9,'B':8,'H':3,'I':4,'G':4,'C':4,'J':4,'F':5}
numhouses={'A':208,'E':161, 'D':138,'B':104,'H':52,'I':48,'G':44,'C':34,'J':26,'F':20}
labels ={}
for s, f in floors.items():
    u = numhouses[s]
    labels[s] = s +'('+str(u)+'/'+str(f)+')'
print(labels)
df.loc[:, 'label'] = df.loc[:, 'site'].apply(lambda x: labels[x])

{'A': 'A(208/12)', 'E': 'E(161/7)', 'D': 'D(138/9)', 'B': 'B(104/8)', 'H': 'H(52/3)', 'I': 'I(48/4)', 'G': 'G(44/4)', 'C': 'C(34/4)', 'J': 'J(26/4)', 'F': 'F(20/5)'}


In [14]:

max_y_value = 3000


for site in [s for s in sites if s in df['site'].values]:
    for pimp in ['8c']:#, '10c', '12c']:
        
        title = "site " + site + ' parent import ' +pimp                   
        df['parent_ret'] = df['parent'].apply(lambda x: x[6:] if 'NULL' not in x else x)
 
        # Slice for parent tariff
        # -----------------------
        df['pimp'] = df['parent_ret'].apply(lambda x : x[0:3] )
        df['pfit'] = df['parent_ret'].apply(lambda x : x[-6:] if 'FIT' in x else x )
        p_slice = df['pimp'] == pimp

        dfx= df.copy()[p_slice]
        #print(dfx.parent)
        # Get rid of duplicates - assumed irrelevant as only variation is internal tariffs
        # --------------------------------------------------------------------------------
        dfx.loc[:,'kwp_str'] = dfx.loc[:,'kwp_unit'].apply(lambda x : str(round(x,2)))
        dfx['combined'] =  dfx['arrangement'] + '_'+ dfx['kwp_str'] + '_'+ dfx['en_capex_id'].fillna('')
        dfx['combined'] = dfx['combined'].apply(lambda x : x.replace('_0.0_','_') if '_0.0_' in x else x)

        dfx['all_combined'] = dfx['combined'] + dfx['parent_ret'].fillna('')
        dfx = dfx.drop_duplicates('all_combined')




In [17]:
dfx.index = [dfx.pimp,dfx.combined]
dfu = dfx['$_unit'].unstack()
dfv = dfu.transpose()
dfv

pimp,8c
combined,
en_capex_4,1410.212435
en_capex_5,1570.823401
en_capex_6,1838.508345
en_pv_0.5_capex_4,1538.939148
en_pv_0.5_capex_5,1699.550114
en_pv_0.5_capex_6,1967.235058
en_pv_1.0_capex_4,1673.824088
en_pv_1.0_capex_5,1834.435054
en_pv_1.0_capex_6,2102.119998


In [ ]:

        # ----------------------
        # reindex and stack data
        # ----------------------
        dfx.index = [dfx.pimp,dfx.combined]
        dfu = dfx['$_unit'].unstack()
    
        # this is not quite right - need to revist the unstacking 


        # --------------------------
        # Choose order of categories
        # --------------------------
        #dfu = dfu.loc[[labels[s] for s in sites],:]


        # Plotting set-up parameters
        cmap = mpl.cm.tab10
        alpha=1


        fig, ax = plt.subplots()
        ax = dfu.plot(kind='bar',width = 1, figsize=(15, 10), fontsize=14)
        ax.set_xlabel("Tariff)", fontsize=14)
        ax.set_ylabel("Total Site Costs $ / unit", fontsize=14)
        ax.set_title ("Site Costs $ / Unit for Site "+ labels[site],fontsize=24)

        # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        # Put a legend to the right of the current axis
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5),fontsize=14)

        ax.grid(True)
        ax.set_ylim((0,max_y_value))
        #fig.show()
        pname = study_name +'site_'+site+'.png'
        plotFile = os.path.join(self.plot_path,pname)
        plt.savefig(plotFile,dpi=1000)





In [ ]:
#df['parent'] = df['parent'].apply(lambda x: x[6:] if 'NULL' not in x else x)
df['parent'] 


In [ ]:
sites